

---

# **Chapter 4: Simplifying Object Creation**

## **Opening Context**
Object creation is the most common source of coupling in software systems. When a class instantiates another class using the `new` operator, it creates a concrete dependency that violates the Dependency Inversion Principle. Creational patterns abstract the instantiation process, allowing systems to remain independent of how their objects are created, composed, and represented. This chapter examines the Factory Method and Abstract Factory patterns—two approaches to decoupling creation from usage, satisfying both the Open/Closed Principle (OCP) and Dependency Inversion Principle (DIP) established in Chapter 2.

---

## **4.1 Factory Method: Defining an Interface for Creation**

### **Intent**
*Define an interface for creating an object, but let subclasses decide which class to instantiate. Factory Method lets a class defer instantiation to subclasses.*

### **The Problem**
Consider a logistics application that initially only handles transportation by truck. The `Logistics` class directly instantiates `Truck` objects:

```typescript
// ❌ Direct instantiation violates DIP
class Logistics {
    planDelivery(): void {
        const truck = new Truck(); // Concrete dependency!
        truck.deliver();
    }
}
```

When the business requires sea logistics, we must modify `Logistics` to instantiate `Ship` objects, violating OCP. Worse, if we need air logistics later, we modify the class again. The creator class is tightly coupled to specific product classes.

### **The Solution: Factory Method Structure**

The Factory Method pattern introduces an abstraction:

1.  **Product Interface**: Declares the interface of objects the factory method creates.
2.  **Concrete Products**: Implement the Product interface.
3.  **Creator (Abstract Class)**: Declares the factory method returning a Product object.
4.  **Concrete Creators**: Override the factory method to return Concrete Product instances.

### **Implementation**

```typescript
/**
 * PRODUCT INTERFACE
 * Defines the contract all transport modes must fulfill.
 * This abstraction allows the Creator to work with any product variant.
 */
interface Transport {
    deliver(): void;
    getCost(): number;
}

/**
 * CONCRETE PRODUCT A
 * Implements the Product interface specific to land transport.
 */
class Truck implements Transport {
    private readonly capacity: number = 1000; // kg
    
    deliver(): void {
        console.log(`Delivering ${this.capacity}kg by land in a box`);
    }
    
    getCost(): number {
        return 500; // Base cost for trucking
    }
}

/**
 * CONCRETE PRODUCT B
 * Implements the Product interface specific to sea transport.
 */
class Ship implements Transport {
    private readonly capacity: number = 10000; // kg
    
    deliver(): void {
        console.log(`Delivering ${this.capacity}kg by sea in a container`);
    }
    
    getCost(): number {
        return 2000; // Base cost for shipping
    }
}

/**
 * CREATOR (Abstract Class)
 * Declares the factory method and contains core business logic
 * that works with Product objects.
 */
abstract class Logistics {
    /**
     * The Factory Method - deferred to subclasses.
     * Returns Transport (Product interface), not concrete class.
     */
    abstract createTransport(): Transport;
    
    /**
     * Core business logic using the Product.
     * This method is inherited by all concrete creators and works
     * with any Transport implementation (OCP compliance).
     */
    planDelivery(): void {
        // Call the factory method to create a product
        const transport = this.createTransport();
        
        console.log("Planning delivery route...");
        console.log(`Estimated cost: $${transport.getCost()}`);
        transport.deliver();
    }
    
    /**
     * Additional business logic that demonstrates why we need
     * the factory method rather than just passing Transport to constructor.
     */
    scheduleReturnTrip(): void {
        const transport = this.createTransport();
        console.log("Scheduling return trip using same transport type");
        transport.deliver();
    }
}

/**
 * CONCRETE CREATOR A
 * Overrides factory method to create Truck instances.
 * This is the "decision point" where instantiation logic lives.
 */
class RoadLogistics extends Logistics {
    createTransport(): Transport {
        // Specific instantiation decision for road logistics
        return new Truck();
    }
}

/**
 * CONCRETE CREATOR B
 * Overrides factory method to create Ship instances.
 */
class SeaLogistics extends Logistics {
    createTransport(): Transport {
        return new Ship();
    }
}

/**
 * CONCRETE CREATOR C (Extension without modification)
 * Adding AirLogistics requires no changes to existing classes (OCP).
 */
class Airplane implements Transport {
    deliver(): void {
        console.log("Delivering by air in a cargo hold");
    }
    getCost(): number {
        return 5000;
    }
}

class AirLogistics extends Logistics {
    createTransport(): Transport {
        return new Airplane();
    }
}
```

**Code Explanation**:

- **Why `abstract createTransport()`?** This forces subclasses to make the instantiation decision while allowing the parent class to define the algorithm (`planDelivery`) that uses the product. This is the **Template Method** pattern applied to creation.

- **DIP Compliance**: `Logistics` depends on `Transport` (abstraction), not `Truck` or `Ship` (concretions). The high-level module (`Logistics`) is independent of low-level module details.

- **OCP Compliance**: Adding `AirLogistics` required creating new classes, not modifying `Logistics`, `RoadLogistics`, or `SeaLogistics`.

### **When to Use Factory Method**

Use Factory Method when:
1.  A class can't anticipate the class of objects it must create.
2.  A class wants its subclasses to specify the objects it creates.
3.  Classes delegate responsibility to one of several helper subclasses, and you want to localize the knowledge of which helper subclass is the delegate.

### **Connection to Dependency Injection**

Modern frameworks (Angular, Spring, ASP.NET Core) use Factory Method concepts but prefer **Constructor Injection**:

```typescript
// Modern approach: Dependency Injection (externalized factory)
class ModernLogistics {
    constructor(private transport: Transport) {} // Injected, not created
    
    planDelivery(): void {
        this.transport.deliver();
    }
}

// Factory becomes external (often framework-managed)
class TransportFactory {
    static create(type: 'road' | 'sea'): Transport {
        switch(type) {
            case 'road': return new Truck();
            case 'sea': return new Ship();
            default: throw new Error("Unknown transport");
        }
    }
}

// Composition root (where concrete choices are made)
const transport = TransportFactory.create('road');
const logistics = new ModernLogistics(transport); // DIP satisfied
```

**Key Difference**: Traditional Factory Method uses inheritance (subclasses decide); modern DI uses composition (external configuration decides).

---

## **4.2 Abstract Factory: Families of Related Objects**

### **Intent**
*Provide an interface for creating families of related or dependent objects without specifying their concrete classes.*

### **The Problem**
Consider a cross-platform UI toolkit. You need to create buttons, checkboxes, and text fields that share a common look-and-feel (Windows style vs. Mac style). Using Factory Method for each widget type leads to inconsistent mixing:

```typescript
// ❌ Problem: Could accidentally mix WindowsButton with MacCheckbox
const button = new WindowsButton();
const checkbox = new MacCheckbox(); // Inconsistent UI family!
```

You need to ensure that UI components belong to the same family (platform).

### **The Solution: Abstract Factory Structure**

1.  **Abstract Factory**: Interface declaring creation methods for each product type.
2.  **Concrete Factories**: Implement creation methods for a specific family (e.g., Windows, Mac).
3.  **Abstract Products**: Interfaces for each distinct product type (Button, Checkbox).
4.  **Concrete Products**: Implementations specific to a family (WindowsButton, MacButton).

### **Implementation**

```typescript
/**
 * ABSTRACT PRODUCT INTERFACES
 * Define the contract for each product in the family.
 */
interface Button {
    render(): void;
    onClick(callback: () => void): void;
}

interface Checkbox {
    render(): void;
    toggle(): void;
}

interface TextField {
    render(): void;
    setText(text: string): void;
}

/**
 * ABSTRACT FACTORY
 * Declares interfaces for creating each abstract product.
 * This is the "family contract" - implementations must provide
 * consistent variants of all products.
 */
interface UIFactory {
    createButton(): Button;
    createCheckbox(): Checkbox;
    createTextField(): TextField;
}

/**
 * CONCRETE PRODUCTS: Windows Family
 */
class WindowsButton implements Button {
    render(): void {
        console.log("Rendering Windows-style button (rectangular, gray)");
    }
    onClick(callback: () => void): void {
        console.log("Windows button click handler attached");
    }
}

class WindowsCheckbox implements Checkbox {
    private checked = false;
    
    render(): void {
        console.log("Rendering Windows checkbox (square, checked=" + this.checked + ")");
    }
    toggle(): void {
        this.checked = !this.checked;
        console.log("Windows checkbox toggled");
    }
}

class WindowsTextField implements TextField {
    private text = "";
    
    render(): void {
        console.log("Rendering Windows text field (3D border)");
    }
    setText(text: string): void {
        this.text = text;
        console.log("Windows text set to: " + text);
    }
}

/**
 * CONCRETE PRODUCTS: Mac Family
 * Note different implementations but same interface.
 */
class MacButton implements Button {
    render(): void {
        console.log("Rendering Mac-style button (rounded, glossy)");
    }
    onClick(callback: () => void): void {
        console.log("Mac button click handler attached with animation");
    }
}

class MacCheckbox implements Checkbox {
    private checked = false;
    
    render(): void {
        console.log("Rendering Mac checkbox (rounded switch, on=" + this.checked + ")");
    }
    toggle(): void {
        this.checked = !this.checked;
        console.log("Mac checkbox animated toggle");
    }
}

class MacTextField implements TextField {
    private text = "";
    
    render(): void {
        console.log("Rendering Mac text field (flat design, subtle border)");
    }
    setText(text: string): void {
        this.text = text;
        console.log("Mac text set with smooth transition: " + text);
    }
}

/**
 * CONCRETE FACTORIES
 * Each factory creates a complete family of related products.
 */
class WindowsUIFactory implements UIFactory {
    createButton(): Button {
        return new WindowsButton();
    }
    createCheckbox(): Checkbox {
        return new WindowsCheckbox();
    }
    createTextField(): TextField {
        return new WindowsTextField();
    }
}

class MacUIFactory implements UIFactory {
    createButton(): Button {
        return new MacButton();
    }
    createCheckbox(): Checkbox {
        return new MacCheckbox();
    }
    createTextField(): TextField {
        return new MacTextField();
    }
}

/**
 * CLIENT CODE
 * Works with any factory, guaranteed to get consistent family.
 */
class Application {
    private factory: UIFactory;
    private button: Button;
    private checkbox: Checkbox;
    private textField: TextField;
    
    constructor(factory: UIFactory) {
        this.factory = factory;
        
        // All products created belong to the same family
        this.button = factory.createButton();
        this.checkbox = factory.createCheckbox();
        this.textField = factory.createTextField();
    }
    
    renderUI(): void {
        console.log("--- Rendering Application UI ---");
        this.button.render();
        this.checkbox.render();
        this.textField.render();
    }
    
    simulateInteraction(): void {
        this.button.onClick(() => console.log("Button clicked!"));
        this.checkbox.toggle();
        this.textField.setText("Hello Abstract Factory");
    }
}

/**
 * CONFIGURATION/CREATION LOGIC
 * Determines which factory to use based on environment.
 */
function initializeApplication(osType: 'windows' | 'mac'): Application {
    let factory: UIFactory;
    
    switch(osType) {
        case 'windows':
            factory = new WindowsUIFactory();
            break;
        case 'mac':
            factory = new MacUIFactory();
            break;
        default:
            throw new Error("Unsupported OS");
    }
    
    return new Application(factory);
}

// Usage
const winApp = initializeApplication('windows');
winApp.renderUI();
// Output: All Windows-style components

const macApp = initializeApplication('mac');
macApp.renderUI();
// Output: All Mac-style components (guaranteed consistency)
```

**Code Explanation**:

- **Family Guarantee**: `Application` receives a `UIFactory` and uses it to create all UI components. It cannot accidentally mix Windows and Mac components because the factory guarantees consistency.

- **DIP Compliance**: `Application` depends on `UIFactory`, `Button`, `Checkbox` (abstractions), not `WindowsButton` or `MacButton`.

- **Extensibility**: Adding a Linux UI family requires only creating `LinuxButton`, `LinuxCheckbox`, `LinuxTextField`, and `LinuxUIFactory`. No existing code changes (OCP).

### **Abstract Factory vs. Factory Method**

| Aspect | Factory Method | Abstract Factory |
|--------|---------------|------------------|
| **Focus** | Creating one product | Creating families of related products |
| **Structure** | Inheritance (subclasses) | Composition (object parameter) |
| **Complexity** | Simpler | More complex (more interfaces) |
| **Use Case** | When only one type varies | When multiple related types must vary together |

### **When to Use Abstract Factory**

Use Abstract Factory when:
1.  A system should be independent of how its products are created, composed, and represented.
2.  A system should be configured with one of multiple families of products.
3.  A family of related product objects is designed to be used together, and you need to enforce this constraint.
4.  You want to provide a class library of products, revealing only their interfaces, not their implementations.

### **Real-World Application: Database Access Layer**

```typescript
// Abstract Products
interface Connection {
    connect(): void;
    query(sql: string): any[];
}

interface Transaction {
    begin(): void;
    commit(): void;
    rollback(): void;
}

// Abstract Factory
interface DatabaseFactory {
    createConnection(): Connection;
    createTransaction(): Transaction;
}

// Concrete Factories for different database families
class PostgresFactory implements DatabaseFactory {
    createConnection(): Connection { return new PostgresConnection(); }
    createTransaction(): Transaction { return new PostgresTransaction(); }
}

class MongoFactory implements DatabaseFactory {
    createConnection(): Connection { return new MongoConnection(); }
    createTransaction(): Transaction { return new MongoTransaction(); }
}

// Application uses consistent database family
class DataAccessLayer {
    constructor(private factory: DatabaseFactory) {}
    
    executeQuery(sql: string): any[] {
        const conn = this.factory.createConnection();
        const tx = this.factory.createTransaction();
        
        tx.begin();
        const result = conn.query(sql);
        tx.commit();
        return result;
    }
}
```

---

## **Chapter Summary**
In this chapter, we explored:

1.  **Factory Method**: Delegates instantiation to subclasses via an abstract method. Best when a class cannot anticipate the type of objects it needs to create, or when subclasses should specify the objects. Satisfies OCP by allowing new product types via new subclasses.

2.  **Abstract Factory**: Creates families of related objects. Best when a system must work with multiple product families and must ensure product compatibility. Satisfies DIP by depending on abstract factory and product interfaces.

3.  **Modern Context**: While these patterns remain relevant, Dependency Injection frameworks often externalize the factory responsibility, using composition over inheritance.

**Key Decision Framework**:
- One product type, multiple variants → **Factory Method**
- Multiple related products that must work together → **Abstract Factory**
- Simple creation with no family constraints → **Simple Factory** (not a GoF pattern, but common utility)

---

## **Next Chapter Preview**
**Chapter 5: Managing Complexity and State (Builder, Prototype, Singleton)**

Having covered family creation, we now examine patterns for handling object construction complexity and object lifecycle management. Chapter 5 introduces the **Builder** pattern for constructing complex objects step-by-step (essential for objects with many optional parameters), the **Prototype** pattern for cloning existing objects efficiently, and the **Singleton** pattern—its benefits for global state management and its significant drawbacks for testability that make it controversial in modern development. We will explore why dependency injection has largely replaced Singleton in contemporary architectures.

---



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../1. foundations_of_software_design/3. documentation_and_communication.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='5. managing_complexity_and_state.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
